In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge  
from sklearn.model_selection import train_test_split

In [6]:
data = pd.read_csv('./weatherAUS.csv')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [7]:
data=data.fillna(data.mean())

In [8]:
# data2.interpolate(method ='linear', limit_direction ='forward') 
data.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
Evaporation          0
Sunshine             0
WindGustDir      10326
WindGustSpeed        0
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am             0
Cloud3pm             0
Temp9am              0
Temp3pm              0
RainToday         3261
RainTomorrow      3267
dtype: int64

In [9]:
for i in ['WindGustDir','WindDir9am','WindDir3pm','RainToday','RainTomorrow']:
  # print(data[[i]].value_counts().index[0][0])
  data=data.fillna({i: data[[i]].value_counts().index[0][0]})

In [10]:
# data['RainTomorrow_cat']=data['RainTomorrow'].astype('category').cat.codes

In [11]:
for i in ['WindDir9am',
  'Date',
  'RainTomorrow',
 'Location',
 'WindDir3pm',
 'RainToday',
 'WindGustDir']:
  data[i]=data[i].astype('category').cat.codes

In [12]:
# data.head()

In [17]:
num_cols =['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustSpeed',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm']
cat_cols=['WindDir9am',
 'Date',
 'Location',
 'WindDir3pm',
 'RainToday',
 'WindGustDir']
target_col='RainTomorrow'

cols = num_cols + cat_cols + [target_col]

In [14]:
# for col in cat_cols:
#     print(f"{col} DISTRIBUTION")
#     print(data[col].value_counts())
#     print()

In [15]:
from sklearn.preprocessing import StandardScaler

pca = StandardScaler()
X = pca.fit_transform(data[num_cols + cat_cols])
X

array([[ 1.89446615e-01, -4.53363105e-02, -2.10071794e-01, ...,
         1.36645776e+00, -5.29795450e-01,  1.04522847e+00],
       [-7.53100728e-01,  2.65043084e-01, -2.81649838e-01, ...,
         1.58681337e+00, -5.29795450e-01,  1.25826214e+00],
       [ 1.10901003e-01,  3.49692009e-01, -2.81649838e-01, ...,
         1.58681337e+00, -5.29795450e-01,  1.47129581e+00],
       ...,
       [-1.06728318e+00,  5.18989861e-01, -2.81649838e-01, ...,
         1.36645776e+00, -5.29795450e-01, -1.08510821e+00],
       [-6.90264238e-01,  5.33098015e-01, -2.81649838e-01, ...,
        -1.05745394e+00, -5.29795450e-01,  1.93093799e-01],
       [ 4.25083451e-01,  2.10513377e-14, -2.81649838e-01, ...,
        -1.27780955e+00, -5.29795450e-01,  1.04522847e+00]])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,data[target_col], test_size=0.25,shuffle=False)
y_test

109095    1
109096    0
109097    0
109098    1
109099    1
         ..
145455    0
145456    0
145457    0
145458    0
145459    0
Name: RainTomorrow, Length: 36365, dtype: int8

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score, mean_squared_error, make_scorer

def print_metrics(y_preds, y):
    print(f'R^2: {r2_score(y_preds, y)}')
    print(f'MSE: {mean_squared_error(y_preds, y)}')

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

In [28]:
gbr_grid_search = GridSearchCV(KNeighborsClassifier(), 
                               [{'n_neighbors': [1, 2, 3, 4, 5]}],
                               error_score=make_scorer(mean_squared_error))
gbr_grid_search.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=make_scorer(mean_squared_error),
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [29]:
print(gbr_grid_search.best_params_)
print(gbr_grid_search.best_score_)
print(gbr_grid_search.best_estimator_)

{'n_neighbors': 5}
0.816215225262386
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


In [24]:
gbr_grid_search = GridSearchCV(LogisticRegression(), 
                               {'C': np.arange(1,11)},
                               error_score=make_scorer(mean_squared_error),
                               verbose=False)
gbr_grid_search.fit(X_train, y_train)


GridSearchCV(cv=None, error_score=make_scorer(mean_squared_error),
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=False)

In [25]:
print(gbr_grid_search.best_params_)
print(gbr_grid_search.best_score_)
print(gbr_grid_search.best_estimator_)

{'C': 1}
0.8375544250423943
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [26]:
gbr_grid_search = GridSearchCV(GaussianNB(), 
                               {'var_smoothing': np.arange(1e-25,1e-15,1)},
                               error_score=make_scorer(mean_squared_error),
                               verbose=False)
gbr_grid_search.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=make_scorer(mean_squared_error),
             estimator=GaussianNB(priors=None, var_smoothing=1e-09),
             iid='deprecated', n_jobs=None,
             param_grid={'var_smoothing': array([1.e-25])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=False)

In [27]:
print(gbr_grid_search.best_params_)
print(gbr_grid_search.best_score_)
print(gbr_grid_search.best_estimator_)

{'var_smoothing': 1e-25}
0.8002841560108163
GaussianNB(priors=None, var_smoothing=1e-25)


In [ ]:
# class NaiveBayesClassifier(object):
#     def init(self):  
#         self.__class_freq = defaultdict(lambda:0)
#         self.__feat_freq = defaultdict(lambda:0)


#     def fit(self, X, y):
#         # calculate classes and features frequencies
#         for feature, label in zip(X, y):
#             self.__class_freq[label] += 1
#             for value in feature:
#                 self.__feat_freq[(value, label)] += 1

#         # normalizate values
#         num_samples = len(X)
#         for k in self.__class_freq:
#             self.__class_freq[k] /= num_samples

#         for value, label in self.__feat_freq:
#             self.__feat_freq[(value, label)] /= self.__class_freq[label]

#         return self

In [ ]:
# # def predict(self, X):
#     # return argmin of classes 
#     return min(self.__class_freq.keys(), 
#                key=lambda c : self.__calculate_class_freq(X, c)) 
 
# def __calculate_class_freq(self, X, clss):
#     # calculate frequence for current class
#     freq = - np.log(self.__class_freq[clss])
   
#     for feat in X: 
#         freq += - np.log(self.__feat_freq.get((feat, clss), 10 ** (-7)))
#     return freq

In [ ]:
# model = NaiveBayesClassifier().fit(X_train, y_train)

# predictions = [model.predict(x) for x in X_test]
# print(accuracy_score(predictions, y_test))
